<a href="https://colab.research.google.com/github/PhChLong/MiniProjects/blob/main/decoder_only_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this session, recreate the whole code for decoder-only transformer to make shakespear-like paragraphs

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [12]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-01-07 09:13:05--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2026-01-07 09:13:06 (220 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [13]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 64
num_steps = 256
emb_dim = 384
num_head = 6
num_blocks = 6
dropout = 0.2

In [15]:
#create vocab
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

#stoi and itos
stoi = {ch:i for i, ch in enumerate(vocab)}
itos = {i:ch for i, ch in enumerate(vocab)}

#encoder, decoder
encoder = lambda x: [stoi[i] for i in x]
decoder = lambda y: [itos[i] for i in y]

In [16]:
# Train and test splits
data = torch.tensor(encoder(text), dtype= torch.long)
train_size = int(len(data) * 0.9)
train_data = data[:train_size]
valid_data = data[train_size:]

In [17]:
# data loading
def get_batch(split):
    data = train_data if split == "train" else valid_data
    idx = torch.randint(len(data) - num_steps, (batch_size,))
    x = torch.stack([data[i:i+num_steps] for i in idx])
    y = torch.stack([data[i+1:i+1+num_steps] for i in idx])
    x, y = x.to(device), y.to(device)
    return x, y

# Nhớ lại cách mà transformer hoạt động
đầu tiên input sẽ được embedding + positional embedding  
từ đó vào trong các block attention, mỗi block gồm 4 component  
Nhắc qua về dim một tí:  
B: Batch_size  
T: Time = Num_steps  
C: Channel = dim  
1. `X` : (B, T, C)
2. `K` : (B, T, d)
3. `Q` : (B, T, d)
4. `V` : (B, T, out_dim)  

oke quay lại cách mô hình hoạt động
1. Masked multihead attention: input sẽ được biến thành K, Q, V và score = softmax(Q.Kt + mask).V --> score sẽ có shape (B, T, out_dim)
2. add, layer_norm
3. Positionwise FFN (sẽ đọc lại chi tiết sau)
4. add, layer_norm
5. lặp lại `xn` lần
6. FC

## Singular Head Attention


In [18]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, d_head):
        # d_head: dim of a head --> the input should be (B, T, d_head)
        super().__init__()
        self.key = nn.Linear(emb_dim, d_head, bias = False)
        self.query = nn.Linear(emb_dim, d_head, bias = False)
        self.value = nn.Linear(emb_dim, d_head, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(num_steps, num_steps)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, X, value = -1e9):
        B, T, C = X.shape
        K, Q, V = self.key(X), self.query(X), self.value(X)
        wei = Q @ K.transpose(1, 2) * K.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril == 0, value)
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        wei = wei @ V #(B, T, d_head)
        return wei
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_head):
        super().__init__()
        d_head = emb_dim // num_head
        self.heads = nn.ModuleList([Head(d_head) for _ in range(num_head)])
        self.prj = nn.Linear(num_head * d_head, emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, X):
        out = torch.cat([h(X) for h in self.heads], dim = -1)
        out = self.prj(out)
        out = self.dropout(out)
        return out
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self):
        super().__init__()
        self.ln1 = nn.Linear(emb_dim, emb_dim * 4)
        self.ln2 = nn.Linear(emb_dim * 4, emb_dim)
        self.relu = nn.ReLU()
    def forward(self, X): #(B, T, emb_dim)
        return self.ln2(self.relu(self.ln1(X)))
# class Add_LayerNorm(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layer_norm = nn.LayerNorm()
#     def forward():
#         pass
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self):
        super().__init__()
        self.mha = MultiHeadAttention(num_head)
        self.ff = FeedForward()
        self.ln1 = nn.LayerNorm(emb_dim)
        self.ln2 = nn.LayerNorm(emb_dim)
    def forward(self, X): #(B, T, emb_dim)
        out = X + self.mha(self.ln1(X))
        out = out + self.ff(self.ln2(X))
        return out
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, emb_dim)
        self.position_embedding_table = nn.Embedding(num_steps, emb_dim)
        self.blocks = nn.Sequential(*[Block() for _ in range(num_blocks)])
        self.ln_f = nn.LayerNorm(emb_dim) # final layer norm
        self.lm_head = nn.Linear(emb_dim, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last num_steps tokens
            idx_cond = idx[:, -num_steps:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [19]:
learning_rate = 3e-4
max_iters = 5000
eval_interval = 500
eval_iters = 200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decoder(m.generate(context, max_new_tokens=500)[0].tolist()))


10.788929 M parameters
step 0: train loss 4.1953, val loss 4.1906


In [ ]:
open('more.txt', 'w').write(decoder(m.generate(context, max_new_tokens=10000)[0].tolist()))